In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
# !wget https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/alxlnet-base-2020-04-10.tar.gz
# !tar -zxf alxlnet-base-2020-04-10.tar.gz
!ls alxlnet-base-2020-04-10

config.json			       model.ckpt-300000.index
model.ckpt-300000.data-00000-of-00001  model.ckpt-300000.meta


In [3]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya/master/pretrained-model/alxlnet/config/alxlnet-base_config.json
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya/master/pretrained-model/preprocess/sp10m.cased.v9.model

In [4]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from xlnet import model_utils
from xlnet.prepro_utils import preprocess_text, encode_ids
import pickle
import json
import alxlnet as xlnet
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [5]:
import sentencepiece as spm

sp_model = spm.SentencePieceProcessor()
sp_model.Load('sp10m.cased.v9.model')

def tokenize_fn(text):
    text = preprocess_text(text, lower= False)
    return encode_ids(sp_model, text)

In [6]:
prefix = 'https://f000.backblazeb2.com/file/malay-dataset/tagging/ontonotes5/'

urls = """
augmentation-address-ontonotes5.json
augmentation-event-ontonotes5.json
augmentation-fac-ontonotes5.json
augmentation-gpe-ontonotes5.json
augmentation-language-ontonotes5.json
augmentation-law-ontonotes5.json
augmentation-loc-ontonotes5.json
augmentation-norp-ontonotes5.json
augmentation-org-ontonotes5.json
augmentation-person-ontonotes5.json
augmentation-product-ontonotes5.json
augmentation-work-of-art-ontonotes5.json
ontonotes5-train-test.json
"""
urls = list(filter(None, urls.split('\n')))

import os

# uncomment to download
# for url in urls:
#     print(url)
#     os.system(f'wget {prefix}{url}')

In [7]:
import json

with open('ontonotes5-train-test.json') as fopen:
    data = json.load(fopen)
data.keys()

dict_keys(['train_X', 'train_Y', 'test_X', 'test_Y'])

In [8]:
train_X = data['train_X']
test_X = data['test_X']
train_Y = data['train_Y']
test_Y = data['test_Y']

In [9]:
d = [
    {'Tag': 'OTHER', 'Description': 'other'},
    {'Tag': 'ADDRESS', 'Description': 'Address of physical location.'},
    {'Tag': 'PERSON', 'Description': 'People, including fictional.'},
    {
        'Tag': 'NORP',
        'Description': 'Nationalities or religious or political groups.',
    },
    {
        'Tag': 'FAC',
        'Description': 'Buildings, airports, highways, bridges, etc.',
    },
    {
        'Tag': 'ORG',
        'Description': 'Companies, agencies, institutions, etc.',
    },
    {'Tag': 'GPE', 'Description': 'Countries, cities, states.'},
    {
        'Tag': 'LOC',
        'Description': 'Non-GPE locations, mountain ranges, bodies of water.',
    },
    {
        'Tag': 'PRODUCT',
        'Description': 'Objects, vehicles, foods, etc. (Not services.)',
    },
    {
        'Tag': 'EVENT',
        'Description': 'Named hurricanes, battles, wars, sports events, etc.',
    },
    {'Tag': 'WORK_OF_ART', 'Description': 'Titles of books, songs, etc.'},
    {'Tag': 'LAW', 'Description': 'Named documents made into laws.'},
    {'Tag': 'LANGUAGE', 'Description': 'Any named language.'},
    {
        'Tag': 'DATE',
        'Description': 'Absolute or relative dates or periods.',
    },
    {'Tag': 'TIME', 'Description': 'Times smaller than a day.'},
    {'Tag': 'PERCENT', 'Description': 'Percentage, including "%".'},
    {'Tag': 'MONEY', 'Description': 'Monetary values, including unit.'},
    {
        'Tag': 'QUANTITY',
        'Description': 'Measurements, as of weight or distance.',
    },
    {'Tag': 'ORDINAL', 'Description': '"first", "second", etc.'},
    {
        'Tag': 'CARDINAL',
        'Description': 'Numerals that do not fall under another type.',
    },
]
d = [d['Tag'] for d in d]
d = ['PAD', 'X'] + d
tag2idx = {i: no for no, i in enumerate(d)}
idx2tag = {no: i for no, i in enumerate(d)}

In [10]:
from glob import glob

augmented = glob('augmentation-*-ontonotes5.json')

for f in augmented:
    print(f)
    with open(f) as fopen:
        data = json.load(fopen)
        
    print(len(data.get('train_X', [])), len(data.get('train_Y', [])))
    print(len(data.get('test_X', [])), len(data.get('test_Y', [])))
    
    train_X.extend(data.get('train_X', []))
    train_Y.extend(data.get('train_Y', []))
    test_X.extend(data.get('test_X', []))
    test_Y.extend(data.get('test_Y', []))

augmentation-org-ontonotes5.json
35150 35150
6300 6300
augmentation-fac-ontonotes5.json
17040 17040
4260 4260
augmentation-loc-ontonotes5.json
13040 13040
2460 2460
augmentation-gpe-ontonotes5.json
21060 21060
0 0
augmentation-work-of-art-ontonotes5.json
4020 4020
1020 1020
augmentation-event-ontonotes5.json
1665 1665
0 0
augmentation-person-ontonotes5.json
37883 37883
2530 2530
augmentation-product-ontonotes5.json
7040 7040
1760 1760
augmentation-law-ontonotes5.json
12584 12584
3161 3161
augmentation-language-ontonotes5.json
6860 6860
0 0
augmentation-norp-ontonotes5.json
30660 30660
5940 5940
augmentation-address-ontonotes5.json
57502 57502
15106 15106


In [15]:
from tqdm import tqdm

SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

special_symbols = {
    "<unk>"  : 0,
    "<s>"    : 1,
    "</s>"   : 2,
    "<cls>"  : 3,
    "<sep>"  : 4,
    "<pad>"  : 5,
    "<mask>" : 6,
    "<eod>"  : 7,
    "<eop>"  : 8,
}

VOCAB_SIZE = 32000
UNK_ID = special_symbols["<unk>"]
CLS_ID = special_symbols["<cls>"]
SEP_ID = special_symbols["<sep>"]
MASK_ID = special_symbols["<mask>"]
EOD_ID = special_symbols["<eod>"]

def XY(strings):
    left_train, right_train = strings[0], strings[1]
    X, Y, segments, masks = [], [], [], []
    for i in tqdm(range(len(left_train))):
        left = left_train[i]
        right = right_train[i]
        bert_tokens = []
        y = []
        for no, orig_token in enumerate(left):
            t = tokenize_fn(orig_token)
            bert_tokens.extend(t)
            if len(t):
                y.append(right[no])
            y.extend(['X'] * (len(t) - 1))
        bert_tokens.extend([4, 3])
        segment = [0] * (len(bert_tokens) - 1) + [SEG_ID_CLS]
        input_mask = [0] * len(segment)
        y.extend(['PAD', 'PAD'])
        y = [tag2idx[i] for i in y]
        if len(bert_tokens) != len(y):
            print(i)
        X.append(bert_tokens)
        Y.append(y)
        segments.append(segment)
        masks.append(input_mask)
    return [(X, Y, segments, masks)]

In [16]:
import cleaning
t = cleaning.multiprocessing(train_X, train_Y, XY)
train_X, train_Y, train_segments, train_masks = [], [], [], []
for x, y, s, m in t:
    train_X.extend(x)
    train_Y.extend(y)
    train_segments.extend(s)
    train_masks.extend(m)

100%|██████████| 40812/40812 [00:40<00:00, 996.73it/s] 


In [17]:
t = cleaning.multiprocessing(test_X, test_Y, XY)
test_X, test_Y, test_segments, test_masks = [], [], [], []
for x, y, s, m in t:
    test_X.extend(x)
    test_Y.extend(y)
    test_segments.extend(s)
    test_masks.extend(m)

100%|██████████| 5494/5494 [00:10<00:00, 519.75it/s] 


In [18]:
from sklearn.utils import shuffle

train_X, train_Y, train_masks, train_segments = shuffle(train_X, train_Y, train_masks, train_segments)

In [19]:
kwargs = dict(
      is_training=True,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.1,
      dropatt=0.1,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='alxlnet-base-2020-04-10/config.json')

In [20]:
epoch = 5
batch_size = 32
warmup_proportion = 0.1
num_train_steps = int(len(train_X) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)
print(num_train_steps, num_warmup_steps)

training_parameters = dict(
      decay_method = 'poly',
      train_steps = num_train_steps,
      learning_rate = 2e-5,
      warmup_steps = num_warmup_steps,
      min_lr_ratio = 0.0,
      weight_decay = 0.00,
      adam_epsilon = 1e-8,
      num_core_per_host = 1,
      lr_layer_decay_rate = 1,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.02,
      clip = 1.0,
      clamp_len=-1,)

102030 10203


In [21]:
class Parameter:
    def __init__(self, decay_method, warmup_steps, weight_decay, adam_epsilon, 
                num_core_per_host, lr_layer_decay_rate, use_tpu, learning_rate, train_steps,
                min_lr_ratio, clip, **kwargs):
        self.decay_method = decay_method
        self.warmup_steps = warmup_steps
        self.weight_decay = weight_decay
        self.adam_epsilon = adam_epsilon
        self.num_core_per_host = num_core_per_host
        self.lr_layer_decay_rate = lr_layer_decay_rate
        self.use_tpu = use_tpu
        self.learning_rate = learning_rate
        self.train_steps = train_steps
        self.min_lr_ratio = min_lr_ratio
        self.clip = clip
        
training_parameters = Parameter(**training_parameters)

In [22]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.float32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.lengths = tf.count_nonzero(self.X, 1)
        self.maxlen = tf.shape(self.X)[1]
        
        xlnet_model = xlnet.XLNetModel(
            xlnet_config=xlnet_config,
            run_config=xlnet_parameters,
            input_ids=tf.transpose(self.X, [1, 0]),
            seg_ids=tf.transpose(self.segment_ids, [1, 0]),
            input_mask=tf.transpose(self.input_masks, [1, 0]))
        output_layer = xlnet_model.get_sequence_output()
        output_layer = tf.transpose(output_layer, [1, 0, 2])
        
        logits = tf.layers.dense(output_layer, dimension_output)
        y_t = self.Y
        log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
            logits, y_t, self.lengths
        )
        self.cost = tf.reduce_mean(-log_likelihood)
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        mask = tf.sequence_mask(self.lengths, maxlen = self.maxlen)
        self.tags_seq, tags_score = tf.contrib.crf.crf_decode(
            logits, transition_params, self.lengths
        )
        self.tags_seq = tf.identity(self.tags_seq, name = 'logits')

        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(self.tags_seq, mask)
        mask_label = tf.boolean_mask(y_t, mask)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [23]:
dimension_output = len(tag2idx)
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead



INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>

Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `layer.__call__` method instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [24]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [25]:
tvars = tf.trainable_variables()
checkpoint = 'alxlnet-base-2020-04-10/model.ckpt-300000'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [26]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from alxlnet-base-2020-04-10/model.ckpt-300000


In [27]:
def merge_sentencepiece_tokens_tagging(x, y):
    new_paired_tokens = []
    n_tokens = len(x)
    rejected = ['<cls>', '<sep>']

    i = 0

    while i < n_tokens:

        current_token, current_label = x[i], y[i]
        if not current_token.startswith('▁') and current_token not in rejected:
            previous_token, previous_label = new_paired_tokens.pop()
            merged_token = previous_token
            merged_label = [previous_label]
            while (
                not current_token.startswith('▁')
                and current_token not in rejected
            ):
                merged_token = merged_token + current_token.replace('▁', '')
                merged_label.append(current_label)
                i = i + 1
                current_token, current_label = x[i], y[i]
            merged_label = merged_label[0]
            new_paired_tokens.append((merged_token, merged_label))

        else:
            new_paired_tokens.append((current_token, current_label))
            i = i + 1

    words = [
        i[0].replace('▁', '')
        for i in new_paired_tokens
        if i[0] not in ['<cls>', '<sep>']
    ]
    labels = [i[1] for i in new_paired_tokens if i[0] not in ['<cls>', '<sep>']]
    return words, labels

In [29]:
string = 'KUALA LUMPUR: Sempena sambutan Aidilfitri minggu depan, Perdana Menteri Tun Dr Mahathir Mohamad dan Menteri Pengangkutan Anthony Loke Siew Fook menitipkan pesanan khas kepada orang ramai yang mahu pulang ke kampung halaman masing-masing. Dalam video pendek terbitan Jabatan Keselamatan Jalan Raya (JKJR) itu, Dr Mahathir menasihati mereka supaya berhenti berehat dan tidur sebentar  sekiranya mengantuk ketika memandu.'

import re

def entities_textcleaning(string, lowering = False):
    """
    use by entities recognition, pos recognition and dependency parsing
    """
    string = re.sub('[^A-Za-z0-9\-\/() ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    original_string = string.split()
    if lowering:
        string = string.lower()
    string = [
        (original_string[no], word.title() if word.isupper() else word)
        for no, word in enumerate(string.split())
        if len(word)
    ]
    return [s[0] for s in string], [s[1] for s in string]

def parse_X(left):
    bert_tokens = []
    for no, orig_token in enumerate(left):
        t = tokenize_fn(orig_token)
        bert_tokens.extend(t)
    bert_tokens.extend([4, 3])
    segment = [0] * (len(bert_tokens) - 1) + [SEG_ID_CLS]
    input_mask = [0] * len(segment)
    s_tokens = [sp_model.IdToPiece(i) for i in bert_tokens]
    return bert_tokens, segment, input_mask, s_tokens

sequence = entities_textcleaning(string)[1]
parsed_sequence, segment_sequence, mask_sequence, xlnet_sequence = parse_X(sequence)

In [30]:
predicted = sess.run(model.tags_seq,
                feed_dict = {
                    model.X: [parsed_sequence],
                    model.segment_ids: [segment_sequence],
                    model.input_masks: [mask_sequence],
                },
        )[0]
merged = merge_sentencepiece_tokens_tagging(xlnet_sequence, [idx2tag[d] for d in predicted])
list(zip(merged[0], merged[1]))

[('Kuala', 'ADDRESS'),
 ('Lumpur', 'ORG'),
 ('Sempena', 'LAW'),
 ('sambutan', 'ADDRESS'),
 ('Aidilfitri', 'ADDRESS'),
 ('minggu', 'EVENT'),
 ('depan', 'NORP'),
 ('Perdana', 'NORP'),
 ('Menteri', 'OTHER'),
 ('Tun', 'FAC'),
 ('Dr', 'NORP'),
 ('Mahathir', 'OTHER'),
 ('Mohamad', 'PRODUCT'),
 ('dan', 'FAC'),
 ('Menteri', 'FAC'),
 ('Pengangkutan', 'ADDRESS'),
 ('Anthony', 'EVENT'),
 ('Loke', 'EVENT'),
 ('Siew', 'LAW'),
 ('Fook', 'EVENT'),
 ('menitipkan', 'EVENT'),
 ('pesanan', 'EVENT'),
 ('khas', 'FAC'),
 ('kepada', 'X'),
 ('orang', 'PRODUCT'),
 ('ramai', 'PRODUCT'),
 ('yang', 'PRODUCT'),
 ('mahu', 'OTHER'),
 ('pulang', 'FAC'),
 ('ke', 'ADDRESS'),
 ('kampung', 'NORP'),
 ('halaman', 'TIME'),
 ('masing-masing', 'EVENT'),
 ('Dalam', 'LOC'),
 ('video', 'PAD'),
 ('pendek', 'PAD'),
 ('terbitan', 'EVENT'),
 ('Jabatan', 'EVENT'),
 ('Keselamatan', 'ORDINAL'),
 ('Jalan', 'EVENT'),
 ('Raya', 'EVENT'),
 ('(Jkjr)', 'WORK_OF_ART'),
 ('itu', 'WORK_OF_ART'),
 ('Dr', 'MONEY'),
 ('Mahathir', 'LANGUAGE'),
 ('m

In [31]:
import time

for e in range(epoch):
    lasttime = time.time()
    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = train_X[i : index]
        batch_y = train_Y[i : index]
        batch_masks = train_masks[i : index]
        batch_segments = train_segments[i : index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = pad_sequences(batch_y, padding='post')
        batch_segments = pad_sequences(batch_segments, padding='post', value = 4)
        batch_masks = pad_sequences(batch_masks, padding='post', value = 1)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
                model.segment_ids: batch_segments,
                model.input_masks: batch_masks,
            },
        )
        assert not np.isnan(cost)
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    pbar = tqdm(
        range(0, len(test_X), batch_size), desc = 'test minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = test_X[i : index]
        batch_y = test_Y[i : index]
        batch_masks = test_masks[i : index]
        batch_segments = test_segments[i : index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = pad_sequences(batch_y, padding='post')
        batch_segments = pad_sequences(batch_segments, padding='post', value = 4)
        batch_masks = pad_sequences(batch_masks, padding='post', value = 1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
                model.segment_ids: batch_segments,
                model.input_masks: batch_masks,
            },
        )
        assert not np.isnan(cost)
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (e, train_loss, train_acc, test_loss, test_acc)
    )
    
    predicted = sess.run(model.tags_seq,
                feed_dict = {
                    model.X: [parsed_sequence],
                    model.segment_ids: [segment_sequence],
                    model.input_masks: [mask_sequence],
                },
        )[0]
    merged = merge_sentencepiece_tokens_tagging(xlnet_sequence, [idx2tag[d] for d in predicted])
    print(list(zip(merged[0], merged[1])))

train minibatch loop:   0%|          | 0/20407 [00:00<?, ?it/s]

time taken: 9405.782040596008
epoch: 0, training loss: 4.537313, training acc: 0.979145, valid loss: 4.936748, valid acc: 0.981108

[('Kuala', 'GPE'), ('Lumpur', 'GPE'), ('Sempena', 'OTHER'), ('sambutan', 'EVENT'), ('Aidilfitri', 'EVENT'), ('minggu', 'OTHER'), ('depan', 'OTHER'), ('Perdana', 'OTHER'), ('Menteri', 'OTHER'), ('Tun', 'PERSON'), ('Dr', 'PERSON'), ('Mahathir', 'PERSON'), ('Mohamad', 'PERSON'), ('dan', 'OTHER'), ('Menteri', 'OTHER'), ('Pengangkutan', 'ORG'), ('Anthony', 'PERSON'), ('Loke', 'PERSON'), ('Siew', 'PERSON'), ('Fook', 'PERSON'), ('menitipkan', 'OTHER'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing-masing', 'OTHER'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'OTHER'), ('Jabatan', 'ORG'), ('Keselamatan', 'ORG'), ('Jalan', 'ORG'), ('Raya', 'ORG'), ('(Jk

train minibatch loop:   0%|          | 0/20407 [00:00<?, ?it/s]

time taken: 9398.288618326187
epoch: 1, training loss: 1.231279, training acc: 0.993845, valid loss: 5.231983, valid acc: 0.983887

[('Kuala', 'GPE'), ('Lumpur', 'GPE'), ('Sempena', 'OTHER'), ('sambutan', 'EVENT'), ('Aidilfitri', 'EVENT'), ('minggu', 'DATE'), ('depan', 'DATE'), ('Perdana', 'OTHER'), ('Menteri', 'OTHER'), ('Tun', 'PERSON'), ('Dr', 'PERSON'), ('Mahathir', 'PERSON'), ('Mohamad', 'PERSON'), ('dan', 'OTHER'), ('Menteri', 'OTHER'), ('Pengangkutan', 'ORG'), ('Anthony', 'PERSON'), ('Loke', 'PERSON'), ('Siew', 'PERSON'), ('Fook', 'PERSON'), ('menitipkan', 'OTHER'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing-masing', 'OTHER'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'OTHER'), ('Jabatan', 'ORG'), ('Keselamatan', 'ORG'), ('Jalan', 'ORG'), ('Raya', 'ORG'), ('(Jkjr

train minibatch loop:   0%|          | 14/20407 [00:06<2:34:30,  2.20it/s, accuracy=0.997, cost=0.5]  


KeyboardInterrupt: 

In [32]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'alxlnet-base-entities/model.ckpt')

'alxlnet-base-entities/model.ckpt'

In [36]:
kwargs = dict(
      is_training=False,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='alxlnet-base-2020-04-10/config.json')

In [37]:
dimension_output = len(tag2idx)
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())

INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>


/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [38]:
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'alxlnet-base-entities/model.ckpt')

INFO:tensorflow:Restoring parameters from alxlnet-base-entities/model.ckpt


In [39]:
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p])
        out.append(out_i)
    return out

In [40]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x = test_X[i : index]
    batch_y = test_Y[i : index]
    batch_masks = test_masks[i : index]
    batch_segments = test_segments[i : index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = pad_sequences(batch_y, padding='post')
    batch_segments = pad_sequences(batch_segments, padding='post', value = 4)
    batch_masks = pad_sequences(batch_masks, padding='post', value = 1)
    predicted = pred2label(sess.run(
        model.tags_seq,
        feed_dict = {
            model.X: batch_x,
            model.segment_ids: batch_segments,
            model.input_masks: batch_masks,
        },
    ))
    real = pred2label(batch_y)
    predict_Y.extend(predicted)
    real_Y.extend(real)

validation minibatch loop: 100%|██████████| 2748/2748 [08:31<00:00,  5.37it/s]


In [41]:
temp_real_Y = []
for r in real_Y:
    temp_real_Y.extend(r)
    
temp_predict_Y = []
for r in predict_Y:
    temp_predict_Y.extend(r)

In [42]:
from sklearn.metrics import classification_report
print(classification_report(temp_real_Y, temp_predict_Y, digits = 5))

              precision    recall  f1-score   support

     ADDRESS    0.99949   0.99981   0.99965     93446
    CARDINAL    0.92765   0.91402   0.92078     48255
        DATE    0.95309   0.93386   0.94338    126548
       EVENT    0.88426   0.93241   0.90770      5711
         FAC    0.92367   0.92991   0.92678     27392
         GPE    0.93880   0.95315   0.94592    101357
    LANGUAGE    0.84296   0.90909   0.87478       803
         LAW    0.95472   0.95091   0.95281     24834
         LOC    0.92551   0.92953   0.92751     34538
       MONEY    0.86719   0.87403   0.87060     30032
        NORP    0.95470   0.89518   0.92398     57014
     ORDINAL    0.86582   0.94721   0.90469      6213
         ORG    0.95300   0.93138   0.94206    219533
       OTHER    0.99080   0.99334   0.99206   3553350
         PAD    0.99992   1.00000   0.99996   1292031
     PERCENT    0.96856   0.96851   0.96853     21722
      PERSON    0.94616   0.96716   0.95655    101981
     PRODUCT    0.87820   0

In [43]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'Placeholder_3',
 'model/transformer/r_w_bias',
 'model/transformer/r_r_bias',
 'model/transformer/word_embedding/lookup_table',
 'model/transformer/word_embedding/lookup_table_2',
 'model/transformer/r_s_bias',
 'model/transformer/seg_embed',
 'model/transformer/layer_shared/rel_attn/q/kernel',
 'model/transformer/layer_shared/rel_attn/k/kernel',
 'model/transformer/layer_shared/rel_attn/v/kernel',
 'model/transformer/layer_shared/rel_attn/r/kernel',
 'model/transformer/layer_shared/rel_attn/o/kernel',
 'model/transformer/layer_shared/rel_attn/LayerNorm/gamma',
 'model/transformer/layer_shared/ff/layer_1/kernel',
 'model/transformer/layer_shared/ff/layer_1/bias',
 'model/transformer/layer_shared/ff/layer_2/kernel',
 'model/transformer/layer_shared/ff/layer_2/bias',
 'model/transformer/layer_shared/ff/LayerNorm/gamma',
 'dense/kernel',
 'dense/bias',
 'transitions',
 'logits']

In [44]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [45]:
freeze_graph('alxlnet-base-entities', strings)

INFO:tensorflow:Restoring parameters from alxlnet-base-entities/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 22 variables.
INFO:tensorflow:Converted 22 variables to const ops.
7687 ops in the final graph.


In [48]:
string = 'Kyrgios, 25, membuat pesanan itu kerana menyedaari pelbagai kesukaran menimpa rakyat Australia ekoran perintah kawalan pergerakan yang diumumkan Mac lalu bagi memerangi wabak COVID-19 di negara berkenaan. Pemain tenis ranking ke-40 dunia yang dilahirkan di Canberra itu meminta pengikut dan penyokongnya agar jangan tidur dalam keadaan perut kosong dalam hantaran Instagram yang meraih lebih 92,000 tanda suka.'

sequence = entities_textcleaning(string)[1]
parsed_sequence, segment_sequence, mask_sequence, xlnet_sequence = parse_X(sequence)
predicted = sess.run(model.tags_seq,
                feed_dict = {
                    model.X: [parsed_sequence],
                    model.segment_ids: [segment_sequence],
                    model.input_masks: [mask_sequence],
                },
        )[0]
merged = merge_sentencepiece_tokens_tagging(xlnet_sequence, [idx2tag[d] for d in predicted])
list(zip(merged[0], merged[1]))

[('Kyrgios', 'DATE'),
 ('25', 'DATE'),
 ('membuat', 'OTHER'),
 ('pesanan', 'OTHER'),
 ('itu', 'OTHER'),
 ('kerana', 'OTHER'),
 ('menyedaari', 'OTHER'),
 ('pelbagai', 'OTHER'),
 ('kesukaran', 'OTHER'),
 ('menimpa', 'OTHER'),
 ('rakyat', 'OTHER'),
 ('Australia', 'GPE'),
 ('ekoran', 'OTHER'),
 ('perintah', 'OTHER'),
 ('kawalan', 'OTHER'),
 ('pergerakan', 'OTHER'),
 ('yang', 'OTHER'),
 ('diumumkan', 'OTHER'),
 ('Mac', 'DATE'),
 ('lalu', 'DATE'),
 ('bagi', 'OTHER'),
 ('memerangi', 'OTHER'),
 ('wabak', 'OTHER'),
 ('Covid-19', 'OTHER'),
 ('di', 'OTHER'),
 ('negara', 'OTHER'),
 ('berkenaan', 'OTHER'),
 ('Pemain', 'OTHER'),
 ('tenis', 'OTHER'),
 ('ranking', 'OTHER'),
 ('ke-40', 'ORDINAL'),
 ('dunia', 'OTHER'),
 ('yang', 'OTHER'),
 ('dilahirkan', 'OTHER'),
 ('di', 'OTHER'),
 ('Canberra', 'GPE'),
 ('itu', 'OTHER'),
 ('meminta', 'OTHER'),
 ('pengikut', 'OTHER'),
 ('dan', 'OTHER'),
 ('penyokongnya', 'OTHER'),
 ('agar', 'OTHER'),
 ('jangan', 'OTHER'),
 ('tidur', 'OTHER'),
 ('dalam', 'OTHER'),
 ('kea

In [ ]:
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.set_random_seed(0)

In [ ]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'alxlnet-base-entities/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

inputs = ['Placeholder', 'Placeholder_1', 'Placeholder_2']
outputs = ['transpose_3']

print(pb, inputs)

transformed_graph_def = TransformGraph(input_graph_def, 
                                       inputs,
                                       ['logits'] + outputs, transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())